<a href="https://colab.research.google.com/github/tayfununal/continuous_CNN/blob/main/Autoencoders_continious.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras import layers
import numpy as np
import tensorflow as tf

np.random.seed(8)
tf.random.set_seed(8)
# This is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# This is our input image
input_img = keras.Input(shape=(784,))
# "encoded" is the encoded representation of the input
encoded = layers.Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(784, activation='sigmoid')(encoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)

In [ ]:
# This model maps an input to its encoded representation
encoder = keras.Model(input_img, encoded)

In [ ]:
# This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

In [ ]:
lamdas = 1.2
batch_size = 60000
class CustomMSE(tf.keras.losses.Loss):

    def __init__(self, regularization_factor=0.1, name="custom_mse"):
        super().__init__(name=name)
        self.regularization_factor = regularization_factor

    def call(self, y_true, y_pred):
        # custom cross_entropy = -1/batch_size * tf.reduce_sum(y_true * tf.math.log(y_pred + 10e-15))
        binary_entropy_loss = keras.losses.BinaryCrossentropy()(y_true, y_pred)
        reg = lamdas * (1/batch_size) * tf.reduce_sum(abs(y_pred - decoder(X_train_with_epsilon)))

        return binary_entropy_loss + reg

In [ ]:
autoencoder.compile(optimizer='adam', loss=CustomMSE())

In [ ]:
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

In [ ]:
# Adding epsilon
tf.random.set_seed(8)
epsilon = np.array(tf.random.normal((60000,28,28), mean=0, stddev= 0.00001))

epsilon = epsilon.reshape((len(epsilon), np.prod(epsilon.shape[1:])))

X_train_with_epsilon = tf.math.add(x_train, epsilon)

print('x shape:', X_train_with_epsilon.shape)
print('Number of images in x', X_train_with_epsilon.shape[0])


In [ ]:
autoencoder.fit(x_train, X_train_with_epsilon,
                epochs=50,
                batch_size=60000,
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
# Encode and decode some digits
# Note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

In [ ]:
# Use Matplotlib (don't ask)
import matplotlib.pyplot as plt

n = 20  # How many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()